In [ ]:
from IPython.display import HTML, display
import os
import datetime
import numpy as np
import math
import pandas as pd
from scipy import stats
from scipy.stats import ttest_ind, chisquare
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from math import cos, asin, sqrt
from time import time
sns.set(color_codes=True)
from datetime import datetime as dt

# pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 300)
# pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_seq_items', 999)
import plotly.graph_objects as go

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide();
        $('.debug').closest(".output_wrapper").hide();
    } else {
        $('div.input').show();
        $('.debug').closest(".output_wrapper").show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle code view"></form>''')

In [ ]:
# df_all = pd.read_csv('yellow_tripdata_2019-06.csv', usecols = ['tpep_pickup_datetime', 'total_amount'], parse_dates=['tpep_pickup_datetime'])
df_all = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-06.csv')
# https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
# https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-06.csv

In [ ]:
outliers_limit = df_all.total_amount.quantile(0.999)

In [ ]:
df_all = df_all[df_all.total_amount.between(0,outliers_limit, inclusive=False)]

In [ ]:
df_all['date'] = df_all['tpep_pickup_datetime'].dt.normalize()

In [ ]:
df_all['week'] = df_all['tpep_pickup_datetime'].dt.week

In [ ]:
# remove not june data
df_all = df_all[df_all.date.between('2019-06-01', '2019-06-30', inclusive=True)]

In [ ]:
# all bootstrapping will be done on subset of data

df = df_all.sample(frac=0.05).reset_index(drop=True).rename(columns={'total_amount':'metric'})
df.shape

# Estimate the mean μ of the distribution and give 95% bootstrap confidence interval.


In [ ]:
obs_mean = df.metric.mean()


In [ ]:
# generate  bootstrap samples, each of size df, compute mean for each

n_bootstraps = 100

# empirical (or pivotal)

def empirical_bootstrap(series, obs_metric, aggfunc = 'mean', return_bounds=True, n_bootstraps = 100, random=100, 
                        verbose=False):
    bootstrap_metrics = []
    for random in range(n_bootstraps):
        bootstrap = series.sample(n=series.shape[0], replace=True, random_state=random)
        bootstrap_metric = bootstrap.agg(aggfunc)
        bootstrap_metrics.append(bootstrap_metric)

    bootstrap_metrics = np.array(bootstrap_metrics)
    # compute δ∗ = x∗ − x for each bootstrap sample (i.e. each column) and sort them from smallest to biggest:
    diff = bootstrap_metrics - obs_metric
    diff.sort()
    if verbose:
        plt.hist(diff)
        plt.show()
        
    percentile_upper = np.percentile(diff, 2.5)
    percentile_lower = np.percentile(diff, 97.5)
#     print(percentile_upper, percentile_lower)
    empirical_lower = obs_metric - percentile_lower
    empirical_upper = obs_metric - percentile_upper
    if return_bounds:
        return(empirical_lower, empirical_upper)
    else:
        return(percentile_upper*(-1), percentile_lower)

In [ ]:
def percentile_bootstrap(series, obs_metric, aggfunc = 'mean', return_bounds=True, n_bootstraps = 100, random=100):
    bootstrap_metrics = []
    for random in range(n_bootstraps):
        bootstrap = series.sample(n=series.shape[0], replace=True, random_state=random)
        bootstrap_metric = bootstrap.agg(aggfunc)
        bootstrap_metrics.append(bootstrap_metric)

    bootstrap_metrics = np.array(bootstrap_metrics)
    bootstrap_metrics.sort()
    percentile_lower = np.percentile(bootstrap_metrics, 2.5)
    percentile_upper = np.percentile(bootstrap_metrics, 97.5)
    if return_bounds:
        return(percentile_lower, percentile_upper)
    else:
        return(obs_metric-percentile_lower, percentile_upper-obs_metric)

In [ ]:
def calculate_ci(sample, confidence = 0.95):
    n = sample.count()
    stdev = sample.std()
    test_stat = stats.norm.ppf((confidence + 1)/2)
    standard_error = test_stat * stdev / math.sqrt(n)
    return standard_error


In [ ]:
ci_lower = df.metric.mean() - calculate_ci(df.metric)
ci_upper = df.metric.mean() + calculate_ci(df.metric)


In [ ]:
# normal bootstrap (how we usually do. 
# The only difference that I take mean as mean of bootstrap runs, not observational

def se_for_normal_bootstrap(sample, confidence = 0.95):
    n = sample.count()
    stdev = sample.std()
    test_stat = stats.norm.ppf((confidence + 1)/2)
    standard_error = test_stat * stdev #/ math.sqrt(n) 
#     У Вассермана в формуле нигде не видно, что стандарт еррор надо делить на квадртный корень кол-ва сэмплов. Если писать как в учебнике, то все не так плохо.
    return standard_error

# version of normal bootstrap from Wasserman book
def normal_interval_bootstrap(series, aggfunc = 'mean', n_bootstraps = 100, return_bounds = True, random=100):
    bootstrap_metrics = []
    for random in range(n_bootstraps):
        bootstrap = series.sample(n=series.shape[0], replace=True, random_state=random)
        bootstrap_metric = bootstrap.agg(aggfunc)
        bootstrap_metrics.append(bootstrap_metric)

    bootstrap_metrics = pd.Series(bootstrap_metrics)
    ci_lower = bootstrap_metrics.agg(aggfunc) - se_for_normal_bootstrap(bootstrap_metrics)
    ci_upper = bootstrap_metrics.agg(aggfunc) + se_for_normal_bootstrap(bootstrap_metrics)
    if return_bounds:
        return(bootstrap_metrics.mean(), ci_lower, ci_upper)
    else:
        return(bootstrap_metrics.mean(), se_for_normal_bootstrap(bootstrap_metrics), se_for_normal_bootstrap(bootstrap_metrics))

In [ ]:
def ci_bootstrap(series, aggfunc = 'mean', n_bootstraps = 100, return_bounds = True, random=100):
    bootstrap_metrics = []
    for random in range(n_bootstraps):
        bootstrap = series.sample(n=series.shape[0], replace=True, random_state=random)
        bootstrap_metric = bootstrap.agg(aggfunc)
        bootstrap_metrics.append(bootstrap_metric)

    bootstrap_metrics = pd.Series(bootstrap_metrics)
    ci_lower = bootstrap_metrics.agg(aggfunc) - calculate_ci(bootstrap_metrics)
    ci_upper = bootstrap_metrics.agg(aggfunc) + calculate_ci(bootstrap_metrics)
    if return_bounds:
        return(bootstrap_metrics.mean(), ci_lower, ci_upper)
    else:
        return(bootstrap_metrics.mean(), calculate_ci(bootstrap_metrics), calculate_ci(bootstrap_metrics))

## Compare 3 methods

In [ ]:
population_mean = df_all.total_amount.mean()
methods = ['empirical', 'percentile', 'normal_interval','normal', 'ci']
means = [obs_mean, obs_mean, obs_mean, ci_bootstrap(df.metric)[0], obs_mean]

lower = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = 100)[0],
        percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = 100, random=100)[0],
        normal_interval_bootstrap(df.metric, return_bounds=False, n_bootstraps = 100, random=100)[1],
        ci_bootstrap(df.metric, return_bounds=False)[1],
        calculate_ci(df.metric)]

upper = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = 100)[1],
        percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = 100, random=100)[1],
        normal_interval_bootstrap(df.metric,  return_bounds=False, n_bootstraps = 100, random=100)[2],
        ci_bootstrap(df.metric, return_bounds=False)[2],
        calculate_ci(df.metric)]


In [ ]:
fig = go.Figure(data=go.Scatter( # Bar or Scatter
        y=methods,
        x=means,
        error_x=dict(
            type='data', 
            symmetric=False,
            array=upper,
            arrayminus=lower)
    )
                , layout ={ "title": f'Bootstrap Samples = {n_bootstraps}'
                           ,"width": 800
#                            , "yaxis": {"range": [19.5,20]}
                          }
                           )
# fig.update_layout(
#     shapes=[
#         dict(type="line", xref="x1", yref="y1",
#             x0=population_mean, y0=0, x1=population_mean, y1=4, line_width=4, dash="dashdot")])


fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=population_mean,
            y0=0,
            x1=population_mean,
            y1=5,
            name="Positive",
            line=dict(
                color="LightSeaGreen",
                width=3,dash="dashdot"
            )))

fig.add_trace(go.Scatter(
    x=[population_mean],
    y=[''],
    text=["Population mean"],
    mode="text",
))

fig.update_layout(height=640)
fig.update_layout(showlegend=False)
        
fig.show()

In [ ]:
results = pd.DataFrame([methods, means, lower, upper]).transpose()
results.columns = ['method', 'mean', 'lower', 'upper']
results

In [ ]:
# TO DO
# compare how it decreases with incresing bootstrap numbers
# another approach to take random

## Experiment with bootsrtap sample numbers

In [ ]:
# try different bootsrtap sample numbers
for n_bootstraps in [50, 100, 200, 500, 1000]:

    means = [obs_mean, obs_mean, obs_mean, ci_bootstrap(df.metric)[0], obs_mean]

    lower = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[0],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[0],
            normal_interval_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[1],
             ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[1],
            calculate_ci(df.metric)]

    upper = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[1],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[1],
            normal_interval_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[2],
             ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[2],
            calculate_ci(df.metric)]
    fig = go.Figure(data=go.Scatter( # Bar or Scatter
        y=methods,
        x=means,
            error_x=dict(
                type='data', 
                symmetric=False,
                array=upper,
                arrayminus=lower)
        )
                    , layout ={ "title": f'Bootstrap Samples = {n_bootstraps}'
                               ,"width": 800
                               , "xaxis": {"range": [19.2,20]}
                              }
                               )

    fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=population_mean,
            y0=0,
            x1=population_mean,
            y1=5,
            name="Positive",
            line=dict(
                color="LightSeaGreen",
                width=3,dash="dashdot"
            )))
    
    fig.add_trace(go.Scatter(
        x=[population_mean],
        y=[''],
        text=["Population mean"],
        mode="text",
    ))
    fig.update_layout(height=640)
    fig.update_layout(showlegend=False)
    
    fig.show()
    results = pd.DataFrame([methods, means, lower, upper]).transpose()
    results.columns = ['method', 'mean', 'lower', 'upper']
#     display(results)

## Experiment with different sample size

In [ ]:
# experiment with different sample size
for size in [50, 100, 500, 1000, 10000, 20000]:
    n_bootstraps = 500
    df = df_all.sample(n=size).reset_index(drop=True).rename(columns={'total_amount':'metric'})
    obs_mean = df.metric.mean()
    
    means = [obs_mean, obs_mean, obs_mean, ci_bootstrap(df.metric)[0], obs_mean]

    lower = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[0],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[0],
            normal_interval_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[1],
             ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[1],
            calculate_ci(df.metric)]

    upper = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[1],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[1],
            normal_interval_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[2],
             ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[2],
            calculate_ci(df.metric)]
    fig = go.Figure(data=go.Scatter( # Bar or Scatter
        y=methods,
        x=means,
            error_x=dict(
                type='data', 
                symmetric=False,
                array=upper,
                arrayminus=lower)
        )
                    , layout ={ "title": f'Samples in data = {size}'
                               ,"width": 800
                               , "xaxis": {"range": [13,24]}
                              }
                               )

    fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=population_mean,
            y0=0,
            x1=population_mean,
            y1=5,
            name="Positive",
            line=dict(
                color="LightSeaGreen",
                width=3,dash="dashdot"
            )))
    
    fig.add_trace(go.Scatter(
        x=[population_mean],
        y=[''],
        text=["Population mean"],
        mode="text",
    ))
    fig.update_layout(height=640)
    fig.update_layout(showlegend=False)
    
    fig.show()
    results = pd.DataFrame([methods, means, lower, upper]).transpose()
    results.columns = ['method', 'mean', 'lower', 'upper']
#     display(results)

In [ ]:
# experiment with different sample size
for size in [50, 100, 500, 1000, 5000, 10000, 20000]:
    n_bootstraps = 50
    df = df_all.sample(n=size).reset_index(drop=True).rename(columns={'total_amount':'metric'})
    obs_mean = df.metric.mean()
    
    means = [obs_mean, obs_mean, obs_mean, ci_bootstrap(df.metric)[0], obs_mean]

    lower = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[0],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[0],
            normal_interval_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[1],
             ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[1],
            calculate_ci(df.metric)]

    upper = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[1],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[1],
            normal_interval_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[2],
             ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[2],
            calculate_ci(df.metric)]
    fig = go.Figure(data=go.Scatter( # Bar or Scatter
        y=methods,
        x=means,
            error_x=dict(
                type='data', 
                symmetric=False,
                array=upper,
                arrayminus=lower)
        )
                    , layout ={ "title": f'Samples in data = {size}'
                               ,"width": 800
                               , "xaxis": {"range": [13,24]}
                              }
                               )

    fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=population_mean,
            y0=0,
            x1=population_mean,
            y1=5,
            name="Positive",
            line=dict(
                color="LightSeaGreen",
                width=3,dash="dashdot"
            )))
    
    fig.add_trace(go.Scatter(
        x=[population_mean],
        y=[''],
        text=["Population mean"],
        mode="text",
    ))
    fig.update_layout(height=640)
    fig.update_layout(showlegend=False)
    
    fig.show()
    results = pd.DataFrame([methods, means, lower, upper]).transpose()
    results.columns = ['method', 'mean', 'lower', 'upper']
#     display(results)

In [ ]:
# experiment with different sample size

for size in [50, 100, 500, 1000, 5000, 10000, 20000]:
    for n in [50,100, 500, 1000]:
        n_bootstraps = n
        df = df_all.sample(n=size, random_state=size+2).reset_index(drop=True).rename(columns={'total_amount':'metric'})
        obs_mean = df.metric.mean()

        means = [obs_mean, obs_mean, obs_mean, ci_bootstrap(df.metric)[0], obs_mean]

        lower = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[0],
                percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[0],
                normal_interval_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[1],
                 ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[1],
                calculate_ci(df.metric)]

        upper = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[1],
                percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[1],
                normal_interval_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[2],
                 ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[2],
                calculate_ci(df.metric)]
        fig = go.Figure(data=go.Scatter( # Bar or Scatter
            y=methods,
            x=means,
                error_x=dict(
                    type='data', 
                    symmetric=False,
                    array=upper,
                    arrayminus=lower)
            )
                        , layout ={ "title": f'Samples in data = {size} with {n} bootstraps'
                                   ,"width": 800
#                                    , "xaxis": {"range": [13,24]}
                                  }
                                   )

        fig.add_shape(
            # Line Vertical
            dict(
                type="line",
                x0=population_mean,
                y0=0,
                x1=population_mean,
                y1=5,
                name="Positive",
                line=dict(
                    color="LightSeaGreen",
                    width=3,dash="dashdot"
                )))

        fig.add_trace(go.Scatter(
            x=[population_mean],
            y=[''],
            text=["Population mean"],
            mode="text",
        ))
        fig.update_layout(height=640)
        fig.update_layout(showlegend=False)

        fig.show()
        results = pd.DataFrame([methods, means, lower, upper]).transpose()
        results.columns = ['method', 'mean', 'lower', 'upper']
    #     display(results)

In [ ]:
# [why percentile and empirical is very similar in notebook, add original data distribution]